### Görev 1: Veriyi Hazırlama

* Adım 1: flo_data_20K.csv verisini okuyunuz. 


In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
 
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x : '%.2f' % x)

In [5]:
df_ = pd.read_csv("G:\\Drive'ım\\Yazılım\\Eğitim\\Veri Bilimi\\CRM\\FLO\\flo_data_20k.csv")
df = df_.copy()

* Adım 2: Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve replace_with_thresholds fonksiyonlarını tanımlayınız. 
Not: cltv hesaplanırken frequency değerleri integer olması gerekmektedir.Bu nedenle alt ve üst limitlerini round() ile yuvarlayınız

In [7]:
def outlier_thresholds(dataframe, veriable):
    quartile1 = dataframe[veriable].quantile(0.01)
    quartile3 = dataframe[veriable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [8]:
def replace_with_thresholds(dataframe, veriable):
    low_limit, up_limit = outlier_thresholds(dataframe, veriable)
    dataframe.loc[(dataframe[veriable] < low_limit), veriable] = round(low_limit,0)
    dataframe.loc[(dataframe[veriable] > up_limit), veriable] = round(up_limit,0)

* Adım 3: "order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline", 
"customer_value_total_ever_online" değişkenlerinin aykırı değerleri varsa baskılayanız.

In [9]:
columns = ["order_num_total_ever_offline","order_num_total_ever_online", "customer_value_total_ever_offline", "customer_value_total_ever_online"]

for col in columns :
    replace_with_thresholds(df,col)

* Adım 4: Omnichannel müşterilerin hem online'dan hem de offline platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam
alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz.

In [10]:
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
df["order_num_total"] = df["order_num_total_ever_offline"] + df["order_num_total_ever_online"]

df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,customer_value_total,order_num_total
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN],939.37,5.00
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",2013.55,21.00
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]",585.32,5.00
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]",121.97,2.00
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR],209.98,2.00


* Adım 5: Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [13]:
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

* Görev 2: CLTV Veri Yapısının Oluşturulması

* * Adım 1: Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak alınız.
* * Adım 2: customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı yeni bir cltv dataframe'i oluşturunuz. 
Monetary değeri satın alma başına ortalama değer olarak, recency ve tenure değerleri ise haftalık cinsten ifade edilecek

* * * recency: Son satın alma tarihi - ilk satın alma tarihi. Haftalık. (kullanıcı özelinde)
* * * T: Müşterinin yaşı. Haftalık. (analiz tarihinden ne kadar süre önce ilk satın alma yapılmış)
* * * frequency: tekrar eden toplam satın alma sayısı (frequency>1)
* * * monetary: satın alma başına ortalama kazanç

In [ ]:
today_date = dt.datetime(2021,6,1)

In [23]:
cltv = pd.DataFrame()

cltv["customer_id"] = df["master_id"]
cltv["recency_cltv_weekly"] = ((df["last_order_date"] - df["first_order_date"]).dt.days) / 7
cltv["T_weekly"] = ((today_date - df["first_order_date"]).dt.days) / 7
cltv["frequency"] = df["order_num_total"]
cltv["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"]

cltv.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.71,5.00,187.87
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,225.00,21.00,95.88
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,79.00,5.00,117.06
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,21.00,2.00,60.98
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.57,2.00,104.99


### Görev 3: BG/NBD, Gamma-Gamma Modellerinin Kurulması ve CLTV’nin Hesaplanması


* Adım 1: BG/NBD modelini fit ediniz
* * 3 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_3_month olarak cltv
dataframe'ine ekleyiniz
* *  6 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_6_month olarak cltv
dataframe'ine ekleyiniz.

In [24]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
#kat sayılara uygulanacak olan ceza kat sayısı

In [26]:
bgf.fit(cltv['frequency'],
        cltv['recency_cltv_weekly'],
        cltv['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.40, b: 0.00, r: 3.67>

In [28]:
cltv["exp_sales_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(12,
                                                        cltv["frequency"],
                                                        cltv["recency_cltv_weekly"],
                                                        cltv["T_weekly"])

In [29]:
cltv["exp_sales_6_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(24,
                                                        cltv["frequency"],
                                                        cltv["recency_cltv_weekly"],
                                                        cltv["T_weekly"])

In [30]:
cltv[["exp_sales_3_month","exp_sales_6_month"]]

,exp_sales_3_month,exp_sales_6_month
0,0.97,1.94
1,0.98,1.96
2,0.67,1.34
3,0.70,1.40
4,0.40,0.79
...,...,...
19940,0.49,0.97
19941,0.48,0.96
19942,0.48,0.96
19943,0.61,1.22


* Adım 2: Gamma-Gamma modelini fit ediniz. Müşterilerin ortalama bırakacakları değeri tahminleyip exp_average_value olarak cltv
dataframe'ine ekleyiniz

In [31]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

In [32]:
ggf.fit(cltv['frequency'], cltv['monetary_cltv_avg'])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

In [33]:
cltv["exp_avarage_value"] = ggf.conditional_expected_average_profit(cltv['frequency'],
                                                                    cltv['monetary_cltv_avg'])
cltv.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_avarage_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.71,5.00,187.87,0.97,1.94,193.63
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,225.00,21.00,95.88,0.98,1.96,96.67
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,79.00,5.00,117.06,0.67,1.34,120.97
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,21.00,2.00,60.98,0.70,1.40,67.32
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.57,2.00,104.99,0.40,0.79,114.33


* Adım 3: 6 aylık CLTV hesaplayınız ve cltv ismiyle dataframe'e ekleyiniz. 
* * • Cltv değeri en yüksek 20 kişiyi gözlemleyiniz.

In [34]:
cltv_ = ggf.customer_lifetime_value(bgf,
                                       cltv['frequency'],
                                       cltv['recency_cltv_weekly'],
                                       cltv['T_weekly'],
                                       cltv['monetary_cltv_avg'],
                                       time=6,
                                       freq="W",
                                       discount_rate=0.01)
cltv["cltv"] = cltv_

In [38]:
cltv.sort_values("cltv",ascending=False)[:10]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_avarage_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.86,8.00,4.00,1401.80,1.09,2.18,1449.06,3315.27
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.14,13.29,11.00,758.09,1.96,3.93,767.36,3160.41
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.71,101.14,7.00,1106.47,0.72,1.44,1127.61,1706.17
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.29,74.71,16.00,501.87,1.56,3.12,506.17,1658.96
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.71,67.43,52.00,166.22,4.64,9.29,166.71,1624.87
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.43,34.57,8.00,601.23,1.26,2.52,611.49,1619.07
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.29,9.57,2.00,862.69,0.79,1.58,923.68,1533.51
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.71,13.14,17.00,259.87,2.77,5.54,262.07,1523.31
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.57,58.86,31.00,228.53,3.08,6.15,229.61,1481.99
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.86,15.71,3.00,743.59,0.87,1.74,778.05,1418.29


### Görev 4: CLTV Değerine Göre Segmentlerin Oluşturulması

* Adım 1: 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırınız ve grup isimlerini veri setine ekleyiniz. 


In [41]:
cltv["cltv_segment"] = pd.qcut(cltv["cltv"], 4, labels=["D", "C", "B", "A"])
cltv.head()


,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_avarage_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.71,5.00,187.87,0.97,1.94,193.63,394.58,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,225.00,21.00,95.88,0.98,1.96,96.67,199.22,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,79.00,5.00,117.06,0.67,1.34,120.97,169.91,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,21.00,2.00,60.98,0.70,1.40,67.32,98.65,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.57,2.00,104.99,0.40,0.79,114.33,94.88,D
